In [2]:
from kubernetes import client, config, stream
import re
import concurrent.futures
import time

# Load Kubernetes configuration
config.load_kube_config()

def measure_bandwidth_from_source_to_target(v1, namespace, source_pod, target_pod, test_duration=5):
    """Measure bandwidth from source pod to target pod."""
    source_name = source_pod.metadata.name
    target_ip = target_pod.status.pod_ip
    target_name = target_pod.metadata.name
    source_node = source_pod.spec.node_name
    target_node = target_pod.spec.node_name

    if source_name == target_name:
        return None  # Skip self-tests

    command = ['iperf3', '-c', target_ip, '-t', str(test_duration)]
    try:
        output = stream.stream(v1.connect_get_namespaced_pod_exec,
                               source_name,
                               namespace,
                               command=command,
                               stderr=True,
                               stdin=False,
                               stdout=True,
                               tty=False)
        # Parse the output for bandwidth
        match = re.search(r'(\d+\.?\d*\s[MKG]bits/sec)', output)
        bandwidth = match.group(1) if match else "Parsing Error"

        # Print the result immediately
        print(f"{source_node} -> {target_node}: {bandwidth}")
        return source_node, target_node, bandwidth

    except Exception as e:
        print(f"Error measuring bandwidth from {source_name} to {target_name}: {e}")
        return source_node, target_node, "Error"

def measure_bandwidth(namespace='measure-nodes-bd', max_concurrent_tasks=3, test_duration=5):
    """Measure bandwidth between all pods in the namespace."""
    v1 = client.CoreV1Api()
    pods = v1.list_namespaced_pod(namespace, label_selector="app=bandwidth-measurement").items

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_tasks) as executor:
        futures = [
            executor.submit(measure_bandwidth_from_source_to_target, v1, namespace, src, tgt, test_duration)
            for src in pods for tgt in pods if src.metadata.name != tgt.metadata.name
        ]

        # Collect and print results as they complete
        results = {}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                src_node, tgt_node, bandwidth = result
                if src_node not in results:
                    results[src_node] = {}
                results[src_node][tgt_node] = bandwidth

    return results

if __name__ == '__main__':
    namespace = 'measure-nodes-bd'
    print("Measuring bandwidth...")
    time.sleep(10)  # Allow some time for the DaemonSet to deploy
    results = measure_bandwidth(namespace=namespace, max_concurrent_tasks= 4, test_duration=10)
    print("\nFinal Cross-Node Bandwidth Results:")
    for src_node, targets in results.items():
        for tgt_node, bandwidth in targets.items():
            print(f"{src_node} -> {tgt_node}: {bandwidth}")


Measuring bandwidth...
k8s-worker-3 -> k8s-worker-1: 685 Mbits/sec
k8s-worker-3 -> k8s-worker-9: 711 Mbits/sec
k8s-worker-3 -> k8s-worker-5: 305 Mbits/sec
k8s-worker-3 -> k8s-worker-8: 245 Mbits/sec
k8s-worker-3 -> k8s-worker-6: 439 Mbits/sec
k8s-worker-3 -> k8s-worker-2: 507 Mbits/sec
k8s-worker-3 -> k8s-worker-4: 485 Mbits/sec
k8s-worker-3 -> k8s-worker-7: 342 Mbits/sec
k8s-worker-1 -> k8s-worker-3: 665 Mbits/sec
k8s-worker-1 -> k8s-worker-8: 754 Mbits/sec
k8s-worker-1 -> k8s-worker-9: 667 Mbits/sec
k8s-worker-1 -> k8s-worker-5: 359 Mbits/sec
k8s-worker-1 -> k8s-worker-2: 608 Mbits/sec
k8s-worker-1 -> k8s-worker-6: 560 Mbits/sec
k8s-worker-1 -> k8s-worker-4: 452 Mbits/sec
k8s-worker-1 -> k8s-worker-7: 300 Mbits/sec
k8s-worker-8 -> k8s-worker-3: 442 Mbits/sec
k8s-worker-8 -> k8s-worker-1: 330 Mbits/sec
k8s-worker-8 -> k8s-worker-5: 434 Mbits/sec
k8s-worker-8 -> k8s-worker-9: 411 Mbits/sec
k8s-worker-8 -> k8s-worker-6: 675 Mbits/sec
k8s-worker-8 -> k8s-worker-7: 528 Mbits/sec
k8s-worke